# Chat with PDF page images

**If you're looking or the web application, check the src/ folder.** 

This notebook demonstrates how to convert PDF pages to images and send them to a vision model for inference

## Authenticate to OpenAI

The following code connects to OpenAI, either using an Azure OpenAI account, GitHub models, or local Ollama model. See the README for instruction on configuring the `.env` file.

In [ ]:
import os

import azure.identity
import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_host = os.getenv("OPENAI_HOST")
if openai_host == "local":
    print("Using local OpenAI-compatible API with no key")
    openai_client = openai.OpenAI(api_key="no-key-required", base_url=os.environ["LOCAL_OPENAI_ENDPOINT"])
elif os.getenv("AZURE_OPENAI_KEY_FOR_CHATVISION"):
    # Authenticate using an Azure OpenAI API key
    # This is generally discouraged, but is provided as a convenience
    print("Using Azure OpenAI with key")
    openai_client = openai.AzureOpenAI(
        api_version=os.getenv("AZURE_OPENAI_API_VERSION") or "2024-02-15-preview",
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_key=os.environ["AZURE_OPENAI_KEY_FOR_CHATVISION"],
    )
elif os.getenv("AZURE_OPENAI_ENDPOINT"):
    tenant_id = os.environ["AZURE_TENANT_ID"]
    print("Using Azure OpenAI with Azure Developer CLI credential for tenant id", tenant_id)
    default_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=tenant_id)
    token_provider = azure.identity.get_bearer_token_provider(
        default_credential, "https://cognitiveservices.azure.com/.default"
    )
    openai_client = openai.AzureOpenAI(
        api_version=os.getenv("AZURE_OPENAI_API_VERSION") or "2024-02-15-preview",
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        azure_ad_token_provider=token_provider,
    )

## Convert PDFs to images

In [2]:
!pip install Pillow PyMuPDF


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import pymupdf
from PIL import Image

filename = "plants.pdf"
doc = pymupdf.open(filename)
for i in range(doc.page_count):
    doc = pymupdf.open(filename)
    page = doc.load_page(i)
    pix = page.get_pixmap()
    original_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    original_img.save(f"page_{i}.png")

## Send images to vision model

In [4]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [ ]:
user_content = [{"text": "What plants are listed on these pages?", "type": "text"}]
for i in range(doc.page_count):
    user_content.append({"image_url": {"url": open_image_as_base64(f"page_{i}.png")}, "type": "image_url"})

response = openai_client.chat.completions.create(
    model=os.environ["OPENAI_MODEL"], messages=[{"role": "user", "content": user_content}]
)

print(response.choices[0].message.content)

Here is a consolidated list of the plants shown on the pages (scientific name followed by common name). Varieties/cultivars are included; duplicate size/price entries are listed once.

- Abutilon palmeri — Palmer’s abutilon
- Acer macrophyllum — Bigleaf maple
- Acer negundo — Box-elder
- Achillea millefolium — Yarrow
- Adenostoma fasciculatum — Chamise
- Aesculus californica — California buckeye
- Agrostis pallens — Thingrass
- Alnus rhombifolia — White alder
- Ambrosia chamissonis — Silvery beachweed
- Anemopsis californica — Yerba mansa
- Angelica hendersonii — Coast angelica
- Apocynum cannabinum — Hemp dogbane
- Aquilegia eximia — Serpentine columbine
- Arbutus menziesii — Pacific madrone
- Arctostaphylos edmundsii ‘Carmel Sur’ — Carmel Sur manzanita
- Armeria maritima — Sea thrift
- Artemisia californica — California sagebrush
- Artemisia douglasiana — Mugwort
- Artemisia pycnocephala — Sandhill sage
- Asarum caudatum — Wild ginger
- Asclepias californica — California milkweed
- A